In [1]:
import pathlib

import pandas as pd
import spacy
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer

BASE_DIR = pathlib.Path.cwd().parent.parent.resolve()
BASE_DIR

WindowsPath('C:/Users/Alejandro Feria/Desktop/Unal/8 semestre/pnl/SilkNews')

### Cargar dataset

In [2]:
clean_df = pd.read_csv(f"{BASE_DIR}/src/data/clean_data/noticias_combinadas.csv")
clean_df['text'] = clean_df['text'].astype(str)
print(clean_df.shape)

df = clean_df.sample(frac=0.1, random_state=42).reset_index(drop=True)
print(df.shape)

df.head()

(5568, 2)
(557, 2)


,text,label
0,los socialistas reunen a cerca de cargos en un...,True
1,la candidata del pp a la comunidad de madrid i...,True
2,con el objetivo de paliar las penurias de pers...,True
3,la desaceleracion economica y la normalizacion...,True
4,un fuerte miedo y temor experimento brayan per...,False


In [3]:
# uv run --with pip spacy download es_core_news_sm
nlp = spacy.load("es_core_news_sm")
nlp

## tokenizar y aplicar stemming

In [4]:
stemmer = SnowballStemmer('spanish')
def tokenize(text: str) -> list[str]:
  return " ".join([stemmer.stem(token.text) for token in nlp(text) if not token.is_punct and not token.is_space])

df["texto_limpio"] = df["text"].apply(tokenize)

In [5]:
df.head()

,text,label,texto_limpio
0,los socialistas reunen a cerca de cargos en un...,True,los social reun a cerc de carg en unas jorn de...
1,la candidata del pp a la comunidad de madrid i...,True,la candidat del pp a la comun de madr isabel d...
2,con el objetivo de paliar las penurias de pers...,True,con el objet de pali las penuri de person en r...
3,la desaceleracion economica y la normalizacion...,True,la desaceler econom y la normaliz del merc inm...
4,un fuerte miedo y temor experimento brayan per...,False,un fuert mied y temor experiment bray perez un...


In [6]:
# Bolsa de Palabras, unigramas y bigramas
def bag_of_words_sin_stopwords(texto_limpio: str) -> list[str]:
  vectorizer = CountVectorizer(ngram_range=(1, 2), stop_words=list(nlp.Defaults.stop_words))
  vectorizer.fit_transform([texto_limpio])
  return list(vectorizer.get_feature_names_out())

def bag_of_words_frecuencia(texto_limpio: str) -> list[str]:
    vectorizer = CountVectorizer(ngram_range=(1, 2), stop_words=list(nlp.Defaults.stop_words))
    vectorizer.fit_transform([texto_limpio])

    vocabulario_eliminacion_frecuencia = {}
    # Filtrar el vocabulario para eliminar palabras con frecuencia < 3
    for palabra, indice in vectorizer.vocabulary_.items():
        if vectorizer.vocabulary_[palabra] >= 3:
            vocabulario_eliminacion_frecuencia[palabra] = indice

    return list(vocabulario_eliminacion_frecuencia.keys())

df["sin_stopwords"] = df["texto_limpio"].apply(bag_of_words_sin_stopwords)
df["frecuencia<3"] = df["texto_limpio"].apply(bag_of_words_frecuencia)

In [7]:
df

,text,label,texto_limpio,sin_stopwords,frecuencia<3
0,los socialistas reunen a cerca de cargos en un...,True,los social reun a cerc de carg en unas jorn de...,"[alcald, alcald portavoc, alic, alic reunir, c...","[social, reun, cerc, carg, jorn, trabaj, grup,..."
1,la candidata del pp a la comunidad de madrid i...,True,la candidat del pp a la comun de madr isabel d...,"[aspir, aspir gobern, ayus, ayus denunci, camp...","[candidat, pp, comun, madr, isabel, diaz, denu..."
2,con el objetivo de paliar las penurias de pers...,True,con el objet de pali las penuri de person en r...,"[activ, activ lleg, beneficiari, beneficiari p...","[objet, pali, penuri, person, riesg, exclusion..."
3,la desaceleracion economica y la normalizacion...,True,la desaceler econom y la normaliz del merc inm...,"[anos, anos recuper, bast, bast mir, desaceler...","[desaceler, econom, normaliz, merc, inmobiliar..."
4,un fuerte miedo y temor experimento brayan per...,False,un fuert mied y temor experiment bray perez un...,"[actu, actu chal, adem, adem cobard, ahor, aho...","[fuert, mied, temor, experiment, bray, perez, ..."
...,...,...,...,...,...
552,una de las primeras medidas del ministerio de ...,False,una de las primer med del ministeri de iguald ...,"[aberr, aberr jurid, abomin, abomin cad, abomi...","[med, ministeri, iguald, dirig, iren, monter, ..."
553,en un comunicado enviado a todos los medios el...,True,en un comunic envi a tod los medi el auditor d...,"[abc, abc rectif, admit, admit marz, auditor, ...","[comunic, envi, tod, medi, auditor, referent, ..."
554,el ministerio de justicia solicito ayer a la f...,True,el ministeri de justici solicit ayer a la fisc...,"[activ, activ agenci, agenci, agenci gestacion...","[ministeri, justici, solicit, ayer, fiscali, g..."
555,vox tendra un rol determinante en madrid pese ...,True,vox tendr un rol determin en madr pes a que ha...,"[cosech, cosech hac, determin, determin madr, ...","[vox, tendr, rol, madr, pes, perd, plumaz, eur..."


In [8]:
df.to_csv(f"{BASE_DIR}/src/data/extraccion_caracteristicas/extraccion_caracteristicas.csv", index=False)